## 1. Dataset

`-` MIMIC-IV dataset

* 65000명의 환자, 200000명의 응급실 환자 데이터
* deathtime
* dod: 퇴원 날짜
* 생존시간 데이터셋 구축
* subject_id: 환자 고유 식별 번호 -> 환자별 기록 하나만 존재. 고유키
* hadm_id: 환자별 입원 식별 확인 번호 - 1번만 있음. 처음 입원 시각을 기준으로 책정하므로, 남겨둘 필요가 없음. 삭제할 겁니다.
* first_admit: 제일 처음의 입원 시간
* last_discharge: 마지막 입원 시간
* survival_time
* death_event
* text

> 처음 입원했을 때의 기록지만 남기고 나머지는 다 버렸음

* Censoring rate: 79.53
* Ties: 서로 다른 사람이 같은 생존 시간을 가지는 시점 13,526개. Tie가 되는 Survival Time 중에 유니크한 개수.

`-` MIMIC-IV-Note

* Chief complaint: 병원에 온 불편사항
* History of present illness
* 과거 병력
* 입원 경과기록
* 신체 검진
* 퇴원 진단명

> Unlabeled? Note가 dataset에 text형태로 들어가있다는 건가? 아니면 둘은 따로 있는 건가?

`-` Labeling Strategy (Feature Extract)

* 상용 모델에 자료(*200)를 다 기억하라고 한 뒤, 변수 몇 개를 뽑아서 차트로 내보내라고 함.
> 단순 자연어 처리로 해결할 수 없는 문제인 것 같음? 아니면 그냥 파일을 달라고 하면 안되나? API 수준에서는 잘 안되는 건가?
* 기존 데이터셋에 없었던 정보, 결측치를 보간하기 위해서 필요한 것 같음.
> DPO에서만 Preference Learning을 하는게 아님. SFT에서의 데이터셋도 애초에 인간의 선호도가 반영되어 있어야 함.

## 2. Problem

`-` Ground-Truth Issue. GT 문제

* Classification: Status Prediction. Dead or Alive를 분류.
> LLM에서 C-index 계산을 위해 marginal probability를 계산(Kaplan Meier). conditional probability는 계산 불가능하다는 한계

* C-index 계산 가능한 형태: 두 환자 쌍에 대한 공변량을 제시해주고 누가 더 오래 생존할 것인지를 예측
> 답변으로부터 예측값 추출 어려움 (제로샷)

* Conditional Hazard를 아예 추정하는 것을 목표로 해보자는 결론.
> $\lambda(t~|~X) = p(T = t~|~T > t-1, X)$
>
> 지금까지 이렇게 생존을 했는데, 얘가 생존을 할 거냐? 다음 토큰이 `dead`일 확률로 Hazard를 생각하자.
>
> 모든 시점에서 수행함으로써 Conditional Hazard 산출 가능

* Counting Process Transformation
> unique observed time을 모두 늘여놓고, 관측값 이전과 이후 늘려놓음. 이후는 넣을 수 없으므로 절사. Discreate로 취급
>
> Observed time까지의 hazard 값만 알 수 있다는 문제가 있어서, IBS 계산 시 문제가 발생 -> hazard 자체를 예측하도록 만들자.

`-` Regression: Hazard Prediction

* Synthetic dataset을 통해 true hazard를 계산할 수 있음: Counting process transformation
* IBS 계산 문제를 해결하기 위해 validation, test set에 대해 observed 이후의 행도 생성
> Continuous data

`-` 해결책? Ground Truth가 없음.

* Real에서 가장 큰 문제는 hazard를 알지 못한다는 것.
1. 일단 어떻게든 hazard를 예측한다. (Tabular Data를 이용) Estimated hazard -> 성능에 한계가 있을 수 있음. 여기서 Pair를 비교할 수 있는, 학습을 real에서 할 수 있는 방법을 구성하면 좋을 것 같음. Deep surv... aft... 또는 survival function을 타겟으로 바꿈...
2. Training할 때 Robust한 방법을 사용하면, Test에서의 데이터 도메인이 달라져도 어느정도 잘 따라갈 수 있을 것이다. Advanced method. 쉽지 않겠네.

`-` 다음 미팅...

* 파이프라인 완성 후, SFT를 일단 기초적으로라도 돌려보는 것으로...
* 가능한 부분을 같이 먼저 진행하는 방향으로...